# COGS 108 - Data Checkpoint

# Names

- Nadia Corral
- Jose Deleon
- Christina Tyagi

<a id='research_question'></a>
# Research Question

*How did changing ozone pollution rates from 2014 to 2018 effect the amount of respiratory diseases in the Central Valley?*

# Dataset(s)

- Dataset Name: Daily Ozone
- Link to the dataset: https://aqs.epa.gov/aqsweb/airdata/download_files.html#AQI
- Number of observations: 34675

Each data set provides the daily ozone measurement for many cities in the US over the duration of one year. We are going to combine the datasets in order to get the daily ozone measurements from 2014 to 2018 for cities in the following counties: Butte, Colusa, Glenn, Fresno, Kern, Kings, Madera, Merced, Placer, San Joaquin, Sacramento, Shasta, Solano, Stanislaus, Sutter, Tehama, Tulare, Yolo and Yuba. 

- Dataset Name: Incidence Rate Report for California by County; Lung and Bronchus (All Stages^), 2014-2018
- Link to the dataset: https://gis.cdc.gov/Cancer/USCS/#/StateCounty/
- Number of observations: 19

The data set provides the rate of new lung and bronchus cancers from 2014-2018 for every county in CA. We are going to collect the rate of new lung and and bronchus cancers for the following counties: Butte, Colusa, Glenn, Fresno, Kern, Kings, Madera, Merced, Placer, San Joaquin, Sacramento, Shasta, Solano, Stanislaus, Sutter, Tehama, Tulare, Yolo and Yuba.

- Dataset Name: Emergency department visits due to asthma, Both sexes, All Ages, All Races/Ethnicities, Conventional, Age-adjusted rates per 10,000 Counties
- Link to the dataset: https://trackingcalifornia.org/asthma/query
- Number of observations: 95

The data set provides the amount of emergency visits per year due to asthma for every county in CA. We are going to collect the amount of emergency visits due to asthma from 2014 to 2018 for the following counties: Butte, Colusa, Glenn, Fresno, Kern, Kings, Madera, Merced, Placer, San Joaquin, Sacramento, Shasta, Solano, Stanislaus, Sutter, Tehama, Tulare, Yolo and Yuba.


# Setup

In [1]:
import pandas as pd
import numpy as np

# Data Cleaning

Describe your data cleaning steps here.

In [4]:
lung_cancer = pd.read_csv('https://raw.githubusercontent.com/cgtyagi/Group062data/main/CountyMap.csv')
lung_cancer.head()

,Area,County,Cancer Type,Year,Sex,Age-Adjusted Rate,Case Count,Population
0,California,Mono County,All Types of Cancer,2014-2018,Male and Female,291.3,256,71241
1,California,Sierra County,All Types of Cancer,2014-2018,Male and Female,319.9,84,14907
2,California,Siskiyou County,All Types of Cancer,2014-2018,Male and Female,354.5,1251,217140
3,California,Modoc County,All Types of Cancer,2014-2018,Male and Female,357.7,258,44761
4,California,Tulare County,All Types of Cancer,2014-2018,Male and Female,370.3,7700,2298431
